In [ ]:
!pip install make-spirals

# Support Vector Machines (SVM)
 
## LinearSVC
 
Las Máquinas de Vectores Soporte (*Support Vector Machines*) realizan la clasificación encontrando el hiperplano que maximiza el margen entre las clases presentes en un conjunto de datos. Este concepto, que de entrada puede parecer confuso, es en realidad una idea bastante intuitiva y viene descrita por la siguiente figura:
 
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/SVM_margin.png/1280px-SVM_margin.png" width=500>
 
En ella podemos observar claramente un conjunto de datos compuesto por 19 muestras (número de puntos) descritas por dos características ($x_1$ y $x_2$) que se clasifican en dos clases (azul y verde). El objetivo de cualquier clasificador es encontrar un hiperplano que separe todo el espacio en 2 de tal modo que las muestras de la clase azul queden a un lado de dicho hiperplano y las muestras de la clase verde queden al lado contrario.
 
Existen, por tanto, infinitos hiperplanos que son capaces de realizar esta división sin dejar ninguna muestra mal clasificada. Cuando utilizamos *SVM* lo que haremos será fijar este hiperplano como aquel que tenga mayor margen, es decir, aquel que mantenga la mayor distancia con aquellas muestras que está más cerca de dicho hiperplano. A este hiperplano le denominaremos límite de decisión (*decision boundary*).
 
Esta idea tan básica nos proporciona la definición de **vectores soporte** (*support vectors*) que dan nombre al método. Un vector soporte será aquella muestra que está incorrectamente clasificada o que se encuentra cerca del límite de decisión. En la figura anterior observamos 3 vectores soporte: 2 azules y 1 verde.
 
Entendida la idea conceptual del clasificador, veamos como podemos formalizarla matemáticamente. No olvidemos que estamos trabajando con un problema de clasificación (por ahora lineal) por lo que la ecuación que define la salida de nuestro clasificador será de la forma:
 
$$
\hat{y}_i = b + \vec{w} \cdot \vec{x}_i = b + \sum_{j=1}^{m} w_j \cdot x_{i,j}
$$
 
Siendo $b$ y $\vec{w} = (w_1, w_2, \dots, w_m)$ los parámetros que el modelo debe aprender.
 
Las Maquinas de Vector Soporte realizan una clasificación binaria atendiendo al siguiente criterio:
 
- Si $\hat{y}_i = b + \vec{w} \cdot \vec{x}_i \geq 1$ la muestra pertenece a la clase 0.
- Si $\hat{y}_i = b + \vec{w} \cdot \vec{x}_i \leq -1$ la muestra pertenece a la clase 1.
 
Nótese que los identificadores *clase 0* y *clase 1* son meras etiquetas, nada tienen que ver con los valores límite de $1$ y $-1$ definidos por SVM.
 
Enlazando esto con la explicación conceptual del método y volviendo a la figura previamente presentada, entendemos que los vectores soporte de la clase 0 son los que verifican que $b + \vec{w} \cdot \vec{x}_i = 1$ mientras que los vectores soporte de la clase 1 son los que verifican que $b + \vec{w} \cdot \vec{x}_i = -1$. Identificaremos por tanto dos nuevos hiperplanos: el **hiperplano positivo** como aquel en el que se sitúan los vectores soporte de la clase 0 y el **hiperplano negativo** como aquel en el que se sitúan los vectores soporte de la clase 1.
 
Con dichos hiperplanos podemos encontrar el límite de decisión como aquel hiperplano que equidiste del hiperplano positivo y del hiperplano negativo. Para ello nos apoyaremos en el concepto de **margen** (*margin*) que se encarga de medir la distancia entre los hiperplanos positivo y negativo. El valor del margen vendrá definido por la siguiente ecuación:
 
$$
margin = \frac{2}{||\vec{w}||}
$$
 
Valor que obtenemos si situamos dos muestras $\vec{x}_1$ y $\vec{x}_2$ una en frente de otra sobre los hiperplanos positivo y negativo. Analíticamente:
 
$$
b + \vec{w} \cdot \vec{x}_1 = 1 
$$
 
$$
b + \vec{w} \cdot \vec{x}_2 = -1 
$$
 
$$
(b + \vec{w} \cdot \vec{x}_1) - (b + \vec{w} \cdot \vec{x}_2) = 1 - (-1)
$$
 
$$
\vec{w} \cdot (\vec{x}_1 - \vec{x}_2) = 2
$$
 
$$
margin = \vec{x}_1 - \vec{x}_2 = \frac{2}{||\vec{w}||}
$$
 
La clave del funcionamiento de las Máquinas de Vector Soporte, y su principal diferencia respecto a otros clasificadores lineales como la Regresión Logística, radica en el hecho de que únicamente los vectores soporte tienen impacto en el entrenamiento del modelo. Todas las muestras del conjunto de datos que no sean vectores soporte no condicionan la definición del límite de decisión.
 
Para lograr esto las Máquinas de Vector Soporte se construyen apoyándose en la función de pérdida *Hinge Loss* que podemos observar a continuación:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(6, 5))

hinge = np.vectorize(lambda v: abs(min(v-1,0.0)))

x = np.linspace(-4, 4)
y = hinge(x)

plt.plot(x, y)
plt.xlabel("b+w·x")
plt.ylabel("hinge(b+w·x)")
plt.show()

A menudo, a esta función se le conoce como $cost$ y tiene por objetivo sancionar únicamente a las muestras que se encuentren mal clasificadas o muy cercanas al límite de decisión. Retomemos un concepto explicado anteriormente, si se verifica que $b + \vec{w} \cdot \vec{x}_i \geq 1$ entonces $\vec{x}_i$ pertenecerá a la clase 0. Asumiendo que la gráfica anterior hace referencia a la clase 0, todas las muestras con $b + \vec{w} \cdot \vec{x}_i \geq 1$ tendrán $cost = 0$. Dicho de otro modo, todas las muestras bien clasificadas tendrán $cost = 0$. Por el contrario, a medida que nos alejamos del hiperplano positivo ($b + \vec{w} \cdot \vec{x}_i = 1$) y nos acercamos al límite de decisión ($b + \vec{w} \cdot \vec{x}_i = 0$) el valor de $cost$ aumenta.
 
Entendida esta idea es evidente que necesitaremos contra función $cost$ para la clase 1, quedando definidas por tanto $cost_0$ y $cost_1$ del siguiente modo:

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1,2, figsize=(12, 5))

cost_0 = np.vectorize(lambda v: max(1-v,0.0))
cost_1 = np.vectorize(lambda v: max(1+v,0.0))

x = np.linspace(-4, 4)

axs[0].plot(x, cost_0(x))
axs[0].set_xlabel("b+w·x")
axs[0].set_ylabel("cost_0")

axs[1].plot(x, cost_1(x))
axs[1].set_xlabel("b+w·x")
axs[1].set_ylabel("cost_1")

Matemáticamente podemos definir las funciones anteriores con las siguientes expresiones:
 
$$
\hat{y}_i = b + \vec{w} \cdot \vec{x}_i
$$
 
$$
cost_0(\hat{y}_i) = max(0, 1 - \hat{y}_i)
$$
 
$$
cost_1(\hat{y}_i) = max(0, 1 + \hat{y}_i)
$$
 
Y unificarlas del siguiente modo:
 
$$
cost(\hat{y}_i) = y_i \cdot cost_1(\hat{y}_i) + (1 - y_i) \cdot cost_0(\hat{y}_i)
$$
 
$$
cost(\hat{y}_i) = y_i \cdot max(0, 1 + \hat{y}_i) + (1 - y_i) \cdot max(0, 1 - \hat{y}_i)
$$
 
Si tenemos en cuenta los datos de entrada obtenemos:
 
$$
cost(\vec{x}_i) = y_i \cdot max(0, 1 + b + \vec{w} \cdot \vec{x}_i) + (1 - y_i) \cdot max(0, 1 - b - \vec{w} \cdot \vec{x}_i)
$$
 
Y finalmente podemos definir la **función de coste** de las máquinas de vector soporte añadiendo un término de regularización como:
 
$$
loss = C \left[ \sum_{i=1}^{n} y_i \cdot max(0, 1 + b + \vec{w} \cdot \vec{x}_i) + (1 - y_i) \cdot max(0, 1 - b - \vec{w} \cdot \vec{x}_i) \right] + \frac{1}{2} \sum_{j=1}^{m} w_j^2
$$
 
Donde $n$ representa el número de muestras y $m$ el número de características o *features*. Observamos, además, que se ha añadido un híper-parámetros $C$ al modelo. $C$, al que podemos llamar no-regularización, permite controlar el proceso de aprendizaje: valores grandes de $C$ harán el modelo muy sensible a los *outlayers* mientras que valores bajos de $C$ harán el método muy generalista.

Las Máquinas de Vector Soporte (lineales) se encuentra definidas en `sklearn` en la clase [`sklearn.svm.LinearSVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). Como se observa en su documenta, la clase dispone de un parámetro `C` que se corresponde con el híper-parámetro $C$ que controla la no-regularización.
 
Una vez ajustado el modelo podemos recuperar sus parámetros accediendo a sus atributos `coef_`(que contiene $\vec{w}$) e `intercep_`(que contiene $b$).
 
Veamos un ejemplo.
 
Supongamos el siguiente conjunto de datos:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=100, n_features=2, centers=2, random_state=3)

min = np.amin(X, axis=0)
max = np.amax(X, axis=0)

diff = max - min

min = min - 0.1 * diff
max = max + 0.1 * diff

plt.figure()

plt.xlim(min[0], max[0])
plt.ylim(min[1], max[1])

plt.xlabel('X1')
plt.ylabel('X2')

plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.bwr)

Ajustemos un modelo mediante `LinearSVC`:

In [ ]:
from sklearn.svm import LinearSVC

linear_svm = LinearSVC().fit(X, y)
print("coef_ = " + str(linear_svm.coef_))
print("intercept_ = " + str(linear_svm.intercept_))

Y lo pintamos:

In [ ]:
import numpy as np

plt.figure()

plt.xlim(min[0], max[0])
plt.ylim(min[1], max[1])

plt.xlabel('X1')
plt.ylabel('X2')

plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.bwr)

line = np.linspace(min[0], max[0])
plt.plot(line, -(line * linear_svm.coef_[0][0] + linear_svm.intercept_) / linear_svm.coef_[0][1], color='green', label='Límite de decisión')
plt.plot(line, (1-(line * linear_svm.coef_[0][0] + linear_svm.intercept_)) / linear_svm.coef_[0][1], color='red', linestyle='--', label='Hiperplano positivo')
plt.plot(line, (-1-(line * linear_svm.coef_[0][0] + linear_svm.intercept_)) / linear_svm.coef_[0][1], color='blue', linestyle='--', label='Hiperplano negativo')

plt.legend(framealpha=0.9).set_zorder(20)

plt.show()

La clase `LinearSVC` que hemos utilizado es, aproximadamente, equivalente a la clase [`sklearn.svm.SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) definiendo el parámetro `kernel='linear'`.Estudiaremos esto más adelante, sin embargo, ahora vamos a hacer uso de la clase `SVC` en lugar de `LinearSVC` para poder representar los vectores soporte.

La clase `SVC` tiene un atributo `support_vectors_` que contienen los vectores soporte utilizados para definir el límite de decisión. Veamos dichos puntos con un ejemplo gráfico:

In [ ]:
from sklearn import svm

cluster_std = 1.7  #@param {type: "slider", min: 0.1, max: 2, step: 0.1}

X, y = make_blobs(n_samples=100, n_features=2, centers=2, cluster_std=cluster_std, random_state=3)

min = np.amin(X, axis=0)
max = np.amax(X, axis=0)

diff = max - min

min = min - 0.1 * diff
max = max + 0.1 * diff

plt.figure()

plt.xlim(min[0], max[0])
plt.ylim(min[1], max[1])

plt.xlabel('X1')
plt.ylabel('X2')

plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.bwr, zorder=15)

svc = svm.SVC(kernel='linear').fit(X, y)

line = np.linspace(min[0], max[0])
plt.plot(line, -(line * svc.coef_[0][0] + svc.intercept_) / svc.coef_[0][1], color='green', label='Límite de decisión')
plt.plot(line, (1-(line * svc.coef_[0][0] + svc.intercept_)) / svc.coef_[0][1], color='red', linestyle='--', label='Hiperplano positivo')
plt.plot(line, (-1-(line * svc.coef_[0][0] + svc.intercept_)) / svc.coef_[0][1], color='blue', linestyle='--', label='Hiperplano negativo')

plt.scatter(svc.support_vectors_[:,0], svc.support_vectors_[:,1], marker='o', color='yellow', s=140)

plt.legend(framealpha=0.9).set_zorder(20)

plt.show()

También es posible obtener una salida probabilística del modelo `SVC` estableciendo el párametro `probability=True` en su constructor. Esta variación, en esencia, mide la distancia de una muestra al límite de decisión y le asocia una probabilidad: las muestras alejadas tendrán una probabilidad cercana a 1 de pertenecer a su clase, mientras que las más cercanas tendrán una probabilidad en torno a 0.5. Mediante el método `predict_proba()` podemos recuperar y representar dichas probabilidades.

In [ ]:
cluster_std = 1.7  #@param {type: "slider", min: 0.1, max: 2, step: 0.1}

X, y = make_blobs(n_samples=100, n_features=2, centers=2, cluster_std=cluster_std, random_state=3)

min = np.amin(X, axis=0)
max = np.amax(X, axis=0)

diff = max - min

min = min - 0.1 * diff
max = max + 0.1 * diff

plt.figure()

plt.xlim(min[0], max[0])
plt.ylim(min[1], max[1])

plt.xlabel('X1')
plt.ylabel('X2')

svc = svm.SVC(kernel='linear', probability=True).fit(X, y)

line = np.linspace(min[0], max[0])
plt.plot(line, -(line * svc.coef_[0][0] + svc.intercept_) / svc.coef_[0][1], color='black', label='Límite de decisión')
plt.plot(line, (1-(line * svc.coef_[0][0] + svc.intercept_)) / svc.coef_[0][1], color='pink', linestyle='--', label='Hiperplano positivo')
plt.plot(line, (-1-(line * svc.coef_[0][0] + svc.intercept_)) / svc.coef_[0][1], color='cyan', linestyle='--', label='Hiperplano negativo')

x1, x2 = np.meshgrid(np.linspace(min[0], max[0]), np.linspace(min[1], max[1]))
proba = svc.predict_proba(np.c_[x1.ravel(), x2.ravel()])[:,0]
proba = proba.reshape(x1.shape)
plt.pcolormesh(x1, x2, proba, cmap=plt.cm.bwr_r, vmin=0, vmax=1)

plt.scatter(X[:,0], X[:,1], c=y, edgecolor='black', cmap='bwr')

plt.legend(framealpha=0.9).set_zorder(20)

plt.show()

La explicación anterior hacer se ha centrado en detallar cómo funcionan las máquinas de vectores soporte para clasificación binaria. No obstante, también se pueden emplear para clasificación multiclase. La única diferencia es que en lugar de realizar una única clasificación binaria haremos tantas clasificaciones binarias como clases haya utilizando la estrategia *one-vs-rest* (también llamada *one-vs-all*). Esta estrategia asume que todas las muestras que no pertenecen a la clase evaluada son de la clase contraria.

Ilustremos el resultado con un ejemplo.

Generamos un conjunto de datos con tres clases:

In [ ]:
X, y = make_blobs(centers=3, random_state=42)

min = np.amin(X, axis=0)
max = np.amax(X, axis=0)

diff = max - min

min = min - 0.1 * diff
max = max + 0.1 * diff

plt.figure()

plt.xlim(min[0], max[0])
plt.ylim(min[1], max[1])

plt.xlabel('X1')
plt.ylabel('X2')

plt.scatter(X[:, 0], X[:, 1], c=y, cmap='rainbow')

Ajustamos el modelo:

In [ ]:
linear_svm = LinearSVC().fit(X, y)
print("coef_ = " + str(linear_svm.coef_))
print("intercept_ = " + str(linear_svm.intercept_))

Analizamos el resultado:

In [ ]:
plt.figure()

plt.xlim(min[0], max[0])
plt.ylim(min[1], max[1])

plt.xlabel('X1')
plt.ylabel('X2')

plt.scatter(X[:,0], X[:,1], c=y, cmap='rainbow')

line = np.linspace(min[0], max[0])

plt.plot(line, -(line * linear_svm.coef_[0][0] + linear_svm.intercept_[0]) / linear_svm.coef_[0][1], color=plt.cm.rainbow(0.0))
plt.plot(line, -(line * linear_svm.coef_[1][0] + linear_svm.intercept_[1]) / linear_svm.coef_[1][1], color=plt.cm.rainbow(0.5))
plt.plot(line, -(line * linear_svm.coef_[2][0] + linear_svm.intercept_[2]) / linear_svm.coef_[2][1], color=plt.cm.rainbow(1.0))

## Non-linear SVM
 
En el apartado anterior hemos visto cómo podemos construir una Máquina de Vectores Soporte que separe clases linealmente. Sin embargo, existen infinidad de problemas en los que la separación lineal no es posible. Las Máquinas de Vectores Soporte pueden funcionar con separaciones no lineales haciendo algunas modificaciones.
 
Principalmente debemos modificar la función de coste por una nueva:
 
$$
loss = C \left[ \sum_{i=1}^{n} y_i \cdot max(0, 1 + \theta^T \cdot f(\vec{x}_i)) + (1 - y_i) \cdot max(0, 1 - \theta^T \cdot f(\vec{x}_i)) \right] + \frac{1}{2} \sum_{j=1}^{m} \theta_j^2
$$
 
Como vemos hemos modificado los vectores $\vec{w}$ por vectores $\theta$ y a los valores de los vectores $\vec{x}_i$ se les ha aplicado una función $f$.
 
Está función $f$ hace uso de una **función de Kernel** y permite determinar la similaridad entre una muestra y todas las demás. Si dos muestras son idénticas entonces su similaridad deberá ser 1, mientras que si son opuestas deberá ser 0.

La idea subyacente de las funciones de Kernel consiste en aumentar la dimensión del conjunto de datos para que, al realizar una separación lineal en un espacio de dimensión superior, se muestre una separación no lineal en el espacio original. Ilustremos esto con un sencillo ejemplo. 

Asumamos la siguiente función de Kernel para comparar dos muestras $x_i$ y $x_j$ cuando trabajamos en $\mathbb{R}^2$:

$$
K(x_i,x_j) = (x_i^t \cdot x_j)^2 = (x_{i,1} \cdot x_{j,1} + x_{i,2} \cdot x_{j,2})^2 = x_{i,1}^2 \cdot x_{j,1}^2 + x_{i,2}^2 \cdot x_{j,2}^2 + 2 \cdot x_{i,1} \cdot x_{j,1} \cdot x_{i,2} \cdot x_{j,2}
$$

Esta función de Kernel es equivalente a:

$$
K(x_i,x_j) = (x_i^t \cdot x_j)^2 = (x_{i,1}^2, x_{i,2}^2, \sqrt{2} \cdot x_{i,1} \cdot x_{i,2}) \cdot (x_{j,1}^2, x_{j,2}^2, \sqrt{2} \cdot x_{j,1} \cdot x_{j,2})
$$

O dicho de otro modo, usar la función de Kernel $K(x_i,x_j) = (x_i^t \cdot x_j)^2$ equivale a transformar nuestro espacio en $\mathbb{R}^2$ a un espacio en $\mathbb{R}^3$ en el que trabajar mediante la siguiente transformación:

$$
(x_{i,1}, x_{i,2}) \rightarrow (x_{i,1}^2, x_{i,2}^2, \sqrt{2} \cdot x_{i,1} \cdot x_{i,2})
$$

¿Y por qué es mejor trabajar en $\mathbb{R}^3$ que en $\mathbb{R}^2$? Veámoslo gráficamente.

Supongamos que tenemos el siguiente conjunto de datos:

In [ ]:
import matplotlib.pyplot as plt

from sklearn.datasets import make_circles

X, y = make_circles(n_samples=50, noise=0.1, factor=0.4, random_state=42)

plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.bwr)

plt.show()

Su transformación a $\mathbb{R}^3$ generaría el siguiente conjunto de datos:

In [ ]:
import plotly.graph_objects as go
import numpy as np
from math import sqrt

fig = go.Figure(data=[go.Scatter3d(
    x = X[:,0]**2,
    y = X[:,1]**2,
    z = sqrt(2)*X[:,0]*X[:,1],
    mode = 'markers',
    marker = dict(
        size = 6,
        color = -y,
        colorscale = 'RdBu',
        opacity = 0.8
    )
)])

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0),
                  scene = dict(
                      xaxis = dict(title='x1^2'),
                      yaxis = dict(title='x2^2'),
                      zaxis = dict(title='sqrt(2)*x1*x2')
                  ))
fig.show()

El cual es separable linealmente.

Existen infinidad de funciones de Kernel, denotadas por $K(x_i,x_j)$:

- Lineal: $K(x_i,x_j) = x_i^t \cdot x_j$
- Polinomial: $K(x_i,x_j) = (x_i^t \cdot x_j+ c)^d$ donde $c>0$
- Sigmoidal: $K(x_i,x_j) = tanh(a \cdot (x_i^t \cdot x_j) + b)$ donde $a,b>0$

Sin embargo, la función de Kernel más popular es la ***Gaussian Kernel***, que se encuentra implementada en `sklearn` bajo el acrónimo `'rbf'` (*Radial Basis Fuction Kernel*). Esta función determina la distancia euclídea entre dos vectores suavizándose mediante una distribución normal de desviación típica $\sigma$:
 
$$
K(x_i,x_j) = exp\left(- \frac{||x_i - x_j||^2}{2\sigma^2} \right)
$$
 
Independientemente de la función de Kernel a utilizar, $f(\vec{x}_i)$ devolverá un vector con el resultado de la función de Kernel (i.e. la distancia) de la muestra $i$ con respecto a todas las demás.

\begin{equation}
f(\vec{x}_i) = \begin{pmatrix} K(x_i,x_1) \\ K(x_i,x_2) \\ \vdots \\ K(x_i,x_i) \\ \vdots \\ K(x_i,x_n) \end{pmatrix}
\end{equation}

Intuitivamente, las muestras cercanas a $i$ tendrán $K(x_i,x_j) \approx 1$ y las muestras lejanas tendrá $K(x_i,x_j) \approx 0$.
 
Por consiguiente, $\theta$ pasará a ser un vector de tamaño $n$ (número de muestras) con los parámetros que el modelo debe aprender.

Este clasificador se encuentra implementado en la clase [`sklearn.svm.SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) de la librería `sklearn`.
 
Se destacan dos parámetros tres parámetros de su implementación:
 
- `C` que indica la no-regularización como en `LinearSVC`.
- `Kernel` que permite modificar la función de Kernel. Por defecto `'rbf'`.
- `gamma` que modifica los parámetros de la función de Kernel. En el caso de `'rbf'`, valores altos de `gamma` indican una baja desviación típica y, por tanto, prestar más atención a las muestras más cercanas, mientras que valores bajos de `gamma` indican lo apuesto.
 
Analicemos la importancia de estos parámetros con un ejemplo gráfico:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
import numpy as np  

X, y = make_moons(n_samples=100, noise=0.08, random_state=10)

min = np.amin(X, axis=0)
max = np.amax(X, axis=0)

diff = max - min

min = min - 0.1 * diff
max = max + 0.1 * diff

fig, axs = plt.subplots(4,4, figsize=(20,20))
fig.tight_layout(pad=4.0)

for i, C in enumerate([0.01, 0.04, 0.08, 0.12]):
  for j, gamma in enumerate([0.1,2,3,10]):
    
    clf = svm.SVC(kernel='rbf', gamma=gamma, C=C, probability=True)
    clf.fit(X, y)

    axs[i,j].set_title('SVM, C: ' + format(C) + ', gamma: '+ str(gamma))

    axs[i,j].set_xlabel('X1')
    axs[i,j].set_ylabel('X2')

    axs[i,j].set_xlim(min[0], max[0])
    axs[i,j].set_ylim(min[1], max[1])

    x1, x2 = np.meshgrid(np.linspace(min[0], max[0]), np.linspace(min[1], max[1]))
    proba = clf.predict_proba(np.c_[x1.ravel(), x2.ravel()])[:,0]
    proba = proba.reshape(x1.shape)
    axs[i,j].pcolormesh(x1, x2, proba, cmap='bwr_r', vmin=0, vmax=1)

    axs[i,j].scatter(X[:,0], X[:,1], c=y, edgecolor='black', cmap='bwr')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from make_spirals import make_spirals
import numpy as np  

X, y = make_spirals(n_samples=1000, random_state=10)

min = np.amin(X, axis=0)
max = np.amax(X, axis=0)

diff = max - min

min = min - 0.1 * diff
max = max + 0.1 * diff

fig, axs = plt.subplots(4,4, figsize=(20,20))
fig.tight_layout(pad=4.0)

for i, C in enumerate([0.01, 0.04, 0.08, 0.12]):
  for j, gamma in enumerate([0.01,0.05,0.1,5]):
    
    clf = svm.SVC(kernel='rbf', gamma=gamma, C=C, probability=True)
    clf.fit(X, y)

    axs[i,j].set_title('SVM, C: ' + format(C) + ', gamma: '+ str(gamma))

    axs[i,j].set_xlabel('X1')
    axs[i,j].set_ylabel('X2')

    axs[i,j].set_xlim(min[0], max[0])
    axs[i,j].set_ylim(min[1], max[1])

    x1, x2 = np.meshgrid(np.linspace(min[0], max[0]), np.linspace(min[1], max[1]))
    proba = clf.predict_proba(np.c_[x1.ravel(), x2.ravel()])[:,0]
    proba = proba.reshape(x1.shape)
    axs[i,j].pcolormesh(x1, x2, proba, cmap='bwr_r', vmin=0, vmax=1)

    axs[i,j].scatter(X[:,0], X[:,1], c=y, edgecolor='black', cmap='bwr')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_circles
from sklearn import svm
import numpy as np  

X, y = make_circles(n_samples=300, noise=0.1, factor=0.5, random_state=42)

min = np.amin(X, axis=0)
max = np.amax(X, axis=0)

diff = max - min

min = min - 0.1 * diff
max = max + 0.1 * diff

fig, axs = plt.subplots(4,4, figsize=(20,20))
fig.tight_layout(pad=4.0)

for i, C in enumerate([0.001, 0.02, 0.04, 0.10]):
  for j, gamma in enumerate([0.25,0.50,0.75,1.00]):
    
    clf = svm.SVC(kernel='rbf', gamma=gamma, C=C, probability=True)
    clf.fit(X, y)

    axs[i,j].set_title('SVM, C: ' + format(C) + ', gamma: '+ str(gamma))

    axs[i,j].set_xlabel('X1')
    axs[i,j].set_ylabel('X2')

    axs[i,j].set_xlim(min[0], max[0])
    axs[i,j].set_ylim(min[1], max[1])

    x1, x2 = np.meshgrid(np.linspace(min[0], max[0]), np.linspace(min[1], max[1]))
    proba = clf.predict_proba(np.c_[x1.ravel(), x2.ravel()])[:,0]
    proba = proba.reshape(x1.shape)
    axs[i,j].pcolormesh(x1, x2, proba, cmap='bwr_r', vmin=0, vmax=1)

    axs[i,j].scatter(X[:,0], X[:,1], c=y, edgecolor='black', cmap='bwr')

---

Creado por **Fernando Ortega** (fernando.ortega@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">